# Modules

In [1]:
import json
import os
import pandas as pd
import re
import requests

from bs4 import BeautifulSoup as bs

# Constantes

In [2]:
PATH_DATA = r"C:\Users\ahoun\Documents\Documents\Nuku Data\ETL-real-estate-prices\Data"
PATH_DATA_RAW = rf"{PATH_DATA}\Raw"

# Application

In [3]:
if os.path.exists(PATH_DATA):
    print('Le dossier Data existe déja.')
else: 
    os.mkdir(PATH_DATA)
    print('Le dossier a été créé avec succès.')

Le dossier Data existe déja.


In [4]:
if os.path.exists(PATH_DATA_RAW):
    print('Le dossier Data raw existe déja.')
else: 
    os.mkdir(PATH_DATA_RAW)
    print('Le dossier Data Raw a été créé avec succès.')

Le dossier Data raw existe déja.


## Téléchargement données d'une page

In [5]:
url = "https://www.immoscout24.ch/en/house/buy/city-bern?pn=1&r=40"
response = requests.get(url)

In [6]:
html = response.content

In [7]:
soup = bs(html, "html")

In [8]:
descs = soup.find_all('div', {'class': "Box-cYFBPY Flex-feqWzG MetaBody-iCkzuU JVKAR dCDRxm dUKbXG"})

data = []
    

for i,des in enumerate(descs):
    content = {'id': i,
                'title': None,
                'description': None,
                'room': None,
                'surface': None,
                'price': None
                }
    content['title'] = des.find('h2').get_text()
    
    if des.find('p'):
        content['description'] = des.find('p').get_text()
    else:
        content['description'] = None


    list_reste = des.find('h3').get_text(strip=True).split('CHF')
    if len(list_reste) == 2:
        price = list_reste[1]
        price = price.replace('CHF ', '')
        price = price.replace('.—', '')
        price = price.replace(',', '').strip()
    else:
        price = None
        

    number_room = list_reste[0].split(',')[0]
    number_room = number_room.replace('rooms', '')
    
    surface = list_reste[0].split(',')[1]
    surface = surface.replace('m²', '')
    
    
    content['room'] = number_room.strip()
    content['surface'] = surface.strip()
    content['price'] = price
    data.append(content)

data

[{'id': 0,
  'title': '«Maison contiguë à Albeuve»',
  'description': "Les maisons de LYS, ce sont cinq villas individuelles groupées situées dans le village d'Albeuve sur la Commune de Haut-Intyamon. Un projet d'architectes conçu en bois suisse, au coeur de la verte Gruyère, qui comprend 3 maisons de 3.5 pièces et 2 maisons de 4.5 pièces. Les surfaces habitables vont de 119 à 220 m² selon les objets.Devenez propriétaire de l'une d'elles et appréciez le calme de la campagne et la vue sur les montagnes. Tout cela à 15 minutes de voiture de Bulle, l'une des villes les plus dynamiques de Suisse romande.Le projet pren",
  'room': '5.5',
  'surface': '220',
  'price': None},
 {'id': 1,
  'title': '«Neue Doppeleinfamilenhaus-Hälfte mit 6Zi. + 250 m2 BF für 1,18 Mio Sfr»',
  'description': 'Im idyllischen Aeschi ( SO ) verkaufen wir an ruhiger Lage in einer Sackgasse eine neu erstellte Doppelinfamilienhaus-Hälfte.Im Untergeschoss befindet sich der Technikraum mit Waschen und Trocknen, Keller,

In [9]:
# Nom du fichier de sortie
page = 1
page_path = rf"{PATH_DATA_RAW}/Page_{page}"
if os.path.exists(page_path):
    print(f'Le dossier pour la page {page} existe déja.')
else: 
    os.mkdir(page_path)
    print(f'Le dossier pour la page {page} a été créé avec succès.')
    

nom_fichier = rf"{page_path}/data_page_{page}.json"
if os.path.exists(nom_fichier):
    print(f'Le fichier data_page_{page} existe déja.')
else: 
    with open(nom_fichier, "w", encoding="utf-8") as fichier: # Sauvegarder le dictionnaire en JSON
        json.dump(data, fichier, ensure_ascii=False)

Le dossier pour la page 1 existe déja.
Le fichier data_page_1 existe déja.


In [10]:
df = pd.read_json(nom_fichier)
df

,id,title,description,room,surface,price
0,0,«Maison contiguë à Albeuve»,"Les maisons de LYS, ce sont cinq villas indivi...",5.5,220,NaN
1,1,«Neue Doppeleinfamilenhaus-Hälfte mit 6Zi. + 2...,Im idyllischen Aeschi ( SO ) verkaufen wir an ...,6.5,250,1515120.0
2,2,«Neubau Einfamilienhaus an bevorzugter und zen...,Das freistehende Einfamilienhaus besticht nich...,5.5,190,1348200.0
3,3,«Neubau Einfamilienhaus an bevorzugter und zen...,Das freistehende Einfamilienhaus besticht nich...,5.5,190,1348200.0
4,4,«Exigez le meilleur (Villa 21A)»,Nouvelle promotion de villas mitoyennes au coe...,5.5,210,1784760.0
5,5,«Le meilleur pour votre famille (Villa 21D)»,Nouvelle promotion de villas mitoyennes au coe...,5.5,210,1861800.0
6,6,«À la hauteur de vos espérances (Villa 21C)»,Nouvelle promotion de villas mitoyennes au coe...,5.5,210,1784760.0
7,7,«Une occasion à ne pas laisser passer (Villa 2...,Nouvelle promotion de villas mitoyennes au coe...,5.5,210,1784760.0
8,8,«Une opportunité unique à saisir (Villa 21)»,Nouvelle promotion de villas mitoyennes au coe...,5.5,210,1861800.0
9,9,«Environnement idéal pour l'épanouissement de ...,"Érigé sur une jolie parcelle, ce bien saura vo...",5.5,180,1181280.0


## Téléchargement données de plusieurs pages

In [11]:
page = 1
url = f"https://www.immoscout24.ch/en/house/buy/city-bern?r=40"
response = requests.get(url)


while response.status_code == 200 :
    url = f"https://www.immoscout24.ch/en/house/buy/city-bern?pn={page}&r=40"
    response = requests.get(url)
    html = response.content
    soup = bs(html, "html")
    descs = soup.find_all('div', {'class': "Box-cYFBPY Flex-feqWzG MetaBody-iCkzuU JVKAR dCDRxm dUKbXG"})
    data = []
    
    for i,des in enumerate(descs):
        content = {'id': i,
                    'title': None,
                    'description': None,
                    'room': None,
                    'surface': None,
                    'price': None
                    }
        content['title'] = des.find('h2').get_text()
    
        if des.find('p'):
            content['description'] = des.find('p').get_text()
        else:
            content['description'] = None

        list_reste = des.find('h3').get_text(strip=True).split('CHF')
        if len(list_reste) == 2:
            price = list_reste[1]
            price = price.replace('CHF ', '')
            price = price.replace('.—', '')
            price = price.replace(',', '').strip()
        else:
            price = None
            
        
        list_rooms = list_reste[0].split('rooms')
        if len(list_rooms) == 2:
            liste_room = des.find('h3').get_text(strip=True).split('rooms')
            number_room = liste_room[0].replace('rooms','').strip()
            list_for_surface = list_rooms[1].split('m²')
            surface = list_for_surface[0].replace(',','').strip()
        else:
            if "rooms" in list_rooms:
                number_room = list_rooms.replace('rooms','').strip()       
        
        content['room'] = number_room.strip()
        content['surface'] = surface.strip()
        content['price'] = price
        data.append(content)
        
        
    # Nom du fichier de sortie
    page_path = rf"{PATH_DATA_RAW}/Page_{page}"
    if os.path.exists(page_path):
        print(f'Le dossier pour la page {page} existe déja.')
    else: 
        os.mkdir(page_path)
        print(f'Le dossier pour la page {page} a été créé avec succès.')
        

    nom_fichier = rf"{page_path}/data_page_{page}.json"
    if os.path.exists(nom_fichier):
        print(f'Le fichier data_page_{page} existe déja.\n')
    else: 
        with open(nom_fichier, "w", encoding="utf-8") as fichier:  # Sauvegarder le dictionnaire en JSON
            json.dump(data, fichier, ensure_ascii=False)
            fichier.close()
            
    page += 1

Le dossier pour la page 1 existe déja.
Le fichier data_page_1 existe déja.

Le dossier pour la page 2 existe déja.
Le fichier data_page_2 existe déja.

Le dossier pour la page 3 existe déja.
Le fichier data_page_3 existe déja.

Le dossier pour la page 4 existe déja.
Le fichier data_page_4 existe déja.

Le dossier pour la page 5 existe déja.
Le fichier data_page_5 existe déja.

Le dossier pour la page 6 existe déja.
Le fichier data_page_6 existe déja.

Le dossier pour la page 7 existe déja.
Le fichier data_page_7 existe déja.

Le dossier pour la page 8 existe déja.
Le fichier data_page_8 existe déja.

Le dossier pour la page 9 existe déja.
Le fichier data_page_9 existe déja.

Le dossier pour la page 10 existe déja.
Le fichier data_page_10 existe déja.

Le dossier pour la page 11 existe déja.
Le fichier data_page_11 existe déja.

Le dossier pour la page 12 existe déja.
Le fichier data_page_12 existe déja.

Le dossier pour la page 13 existe déja.
Le fichier data_page_13 existe déja.

Le d

In [12]:
path_page = rf'{PATH_DATA_RAW}\Page_1\data_page_1.json'
df = pd.read_json(path_page)
df

,id,title,description,room,surface,price
0,0,«Maison contiguë à Albeuve»,"Les maisons de LYS, ce sont cinq villas indivi...",5.5,220,NaN
1,1,«Neue Doppeleinfamilenhaus-Hälfte mit 6Zi. + 2...,Im idyllischen Aeschi ( SO ) verkaufen wir an ...,6.5,250,1515120.0
2,2,«Neubau Einfamilienhaus an bevorzugter und zen...,Das freistehende Einfamilienhaus besticht nich...,5.5,190,1348200.0
3,3,«Neubau Einfamilienhaus an bevorzugter und zen...,Das freistehende Einfamilienhaus besticht nich...,5.5,190,1348200.0
4,4,«Exigez le meilleur (Villa 21A)»,Nouvelle promotion de villas mitoyennes au coe...,5.5,210,1784760.0
5,5,«Le meilleur pour votre famille (Villa 21D)»,Nouvelle promotion de villas mitoyennes au coe...,5.5,210,1861800.0
6,6,«À la hauteur de vos espérances (Villa 21C)»,Nouvelle promotion de villas mitoyennes au coe...,5.5,210,1784760.0
7,7,«Une occasion à ne pas laisser passer (Villa 2...,Nouvelle promotion de villas mitoyennes au coe...,5.5,210,1784760.0
8,8,«Une opportunité unique à saisir (Villa 21)»,Nouvelle promotion de villas mitoyennes au coe...,5.5,210,1861800.0
9,9,«Environnement idéal pour l'épanouissement de ...,"Érigé sur une jolie parcelle, ce bien saura vo...",5.5,180,1181280.0
